<a href="https://colab.research.google.com/github/sjtalkar/DP-203-Azure-Data-Engineering-Notes/blob/main/PixieDustAndScala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#https://pixiedust.github.io/pixiedust/install.html
#PixeDustAndScala.ipynb
#https://github.com/pixiedust/pixiedust/blob/master/install/createKernel.py
#https://archive.apache.org/dist/spark/

# Step 1: PIXIEDUST_HOME: /root/pixiedust
# 	Keep y/n [y]? n
# Please enter a PIXIEDUST_HOME location: /content/pixiedust
# Step 2: SPARK_HOME: /content/spark-3.2.0-bin-hadoop3.2
# 	Keep y/n [y]? n
# Step 2: Please enter a SPARK_HOME location: /content/spark-2.3.2
# Directory /content/spark-2.3.2 does not exist
# 	Create y/n [y]? y
# What version would you like to download? 1.6.3, 2.0.2, 2.1.0, 2.2.0, 2.3.2 [2.3.2]: 
# SPARK_HOME will be set to /content/spark-2.3.2/spark-2.3.2-bin-hadoop2.7
# Downloading Spark 2.3.2

### Successful installation
# Step 1: PIXIEDUST_HOME: /root/pixiedust
# 	Keep y/n [y]? y
# Step 2: SPARK_HOME: /content/spark-2.3.2-bin-hadoop2.7
# 	Keep y/n [y]? y
#  22 %
#  45 %
#  68 %
#  91 %
#  100 %
      
# downloaded spark cloudant jar: /root/pixiedust/bin/cloudant-spark-v2.0.0-185.jar
# Step 3: SCALA_HOME: /root/pixiedust/bin/scala
# 	Keep y/n [y]? y
# Directory /root/pixiedust/bin/scala does not contain a valid scala install
# 	Download Scala y/n [y]? y
# SCALA_HOME will be set to /root/pixiedust/bin/scala/scala-2.11.8
# Downloading Scala 2.11

#Since pixiedust will need a spark home, install and set spark home first. Mount the google drive so that the downlaoded version of spark that works with delta lake is the one installed.
#Spark and delta lake are finicky in the sense not all spark and delta lake versions work well together and the errors resulting from the mismatch are not easy to decipher from the error messages
#Now pixiedust only works with these spark versions: What version would you like to download? 1.6.3, 2.0.2, 2.1.0, 2.2.0, 2.3.2 [2.3.2]:

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!tar xf 'gdrive/My Drive/Databricks/spark-2.3.2-bin-hadoop2.7.tgz' 
!pip -q install findspark
!pip install ipython-sql

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.2-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'


In [5]:
!pip install pixiedust

In [6]:
!jupyter pixiedust install

Step 1: PIXIEDUST_HOME: /root/pixiedust
	Keep y/n [y]? y
Step 2: SPARK_HOME: /content/spark-2.3.2-bin-hadoop2.7
	Keep y/n [y]? y
 22 %
 45 %
 68 %
 91 %
 100 %
      
downloaded spark cloudant jar: /root/pixiedust/bin/cloudant-spark-v2.0.0-185.jar
Step 3: SCALA_HOME: /root/pixiedust/bin/scala
	Keep y/n [y]? y
Directory /root/pixiedust/bin/scala does not contain a valid scala install
	Download Scala y/n [y]? y
SCALA_HOME will be set to /root/pixiedust/bin/scala/scala-2.11.8
 3 %
 7 %
 10 %
 14 %
 18 %
 21 %
 25 %
 29 %
 32 %
 36 %
 40 %
 43 %
 47 %
 51 %
 54 %
 58 %
 62 %
 65 %
 69 %
 73 %
 76 %
 80 %
 84 %
 87 %
 91 %
 95 %
 98 %
 100 %
      
Extracting Scala 2.11 to /root/pixiedust/bin/scala
 0 %
 1 %
 3 %
 5 %
 6 %
 8 %
 10 %
 11 %
 13 %
 15 %
 16 %
 18 %
 20 %
 22 %
 23 %
 25 %
 27 %
 28 %
 30 %
 32 %
 33 %
 35 %
 37 %
 38 %
 40 %
 42 %
 44 %
 45 %
 47 %
 49 %
 50 %
 52 %
 54 %
 55 %
 57 %
 59 %
 61 %
 62 %
 64 %
 66 %
 67 %
 69 %
 71 %
 72 %
 74 %
 76 %
 77 %
 79 %
 81 %
 83 %
 84

In [15]:
!jupyter pixiedust list

#pythonwithpixiedustspark23    /root/.local/share/jupyter/kernels/pythonwithpixiedustspark23


Available kernels:
  pythonwithpixiedustspark23    /root/.local/share/jupyter/kernels/pythonwithpixiedustspark23
  ir                            /usr/local/share/jupyter/kernels/ir
  python2                       /usr/local/share/jupyter/kernels/python2
  python3                       /usr/local/share/jupyter/kernels/python3


In [17]:
%%scala

import java.time.{LocalDate}
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit

import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions._
import scala.collection.mutable

def calculateLastday(wrappedArray: mutable.WrappedArray[Row]) :String ={
  val orderedList = wrappedArray.sortWith((x:Row, y:Row) => {
    if(x.getAs[String]("transaction_date") != y.getAs[String]("transaction_date")) {
      x.getAs[String]("transaction_date") < y.getAs[String]("transaction_date")
    } else {
      
      val x_sig = x.getAs[String]("plan_list_price") +
        x.getAs[String]("payment_plan_days") +
        x.getAs[String]("payment_method_id")

      val y_sig = y.getAs[String]("plan_list_price") +
        y.getAs[String]("payment_plan_days") +
        y.getAs[String]("payment_method_id")

      //same plan, always subscribe then unsubscribe
      if(x_sig != y_sig) {
        x_sig > y_sig
      } else {
        if(x.getAs[String]("is_cancel")== "1" && y.getAs[String]("is_cancel") == "1") {
          //multiple cancel, consecutive cancels should only put the expiration date earlier
          x.getAs[String]("membership_expire_date") > y.getAs[String]("membership_expire_date")
        } else if(x.getAs[String]("is_cancel")== "0" && y.getAs[String]("is_cancel") == "0") {
          //multiple renewal, expiration date keeps extending
          x.getAs[String]("membership_expire_date") < y.getAs[String]("membership_expire_date")
        } else {
          //same day same plan transaction: subscription preceeds cancellation
          x.getAs[String]("is_cancel") < y.getAs[String]("is_cancel")
        }
      }
    }
  })
  orderedList.last.getAs[String]("membership_expire_date")
}

def calculateRenewalGap(log:mutable.WrappedArray[Row], lastExpiration: String): Int = {
  val orderedDates = log.sortWith((x:Row, y:Row) => {
    if(x.getAs[String]("transaction_date") != y.getAs[String]("transaction_date")) {
      x.getAs[String]("transaction_date") < y.getAs[String]("transaction_date")
    } else {
      
      val x_sig = x.getAs[String]("plan_list_price") +
        x.getAs[String]("payment_plan_days") +
        x.getAs[String]("payment_method_id")

      val y_sig = y.getAs[String]("plan_list_price") +
        y.getAs[String]("payment_plan_days") +
        y.getAs[String]("payment_method_id")

      //same data same plan transaction, assumption: subscribe then unsubscribe
      if(x_sig != y_sig) {
        x_sig > y_sig
      } else {
        if(x.getAs[String]("is_cancel")== "1" && y.getAs[String]("is_cancel") == "1") {
          //multiple cancel of same plan, consecutive cancels should only put the expiration date earlier
          x.getAs[String]("membership_expire_date") > y.getAs[String]("membership_expire_date")
        } else if(x.getAs[String]("is_cancel")== "0" && y.getAs[String]("is_cancel") == "0") {
          //multiple renewal, expire date keep extending
          x.getAs[String]("membership_expire_date") < y.getAs[String]("membership_expire_date")
        } else {
          //same date cancel should follow subscription
          x.getAs[String]("is_cancel") < y.getAs[String]("is_cancel")
        }
      }
    }
  })

  //Search for the first subscription after expiration
  //If active cancel is the first action, find the gap between the cancellation and renewal
  val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")
  var lastExpireDate = LocalDate.parse(s"${lastExpiration.substring(0,4)}-${lastExpiration.substring(4,6)}-${lastExpiration.substring(6,8)}", formatter)
  var gap = 9999
  for(
    date <- orderedDates
    if gap == 9999
  ) {
    val transString = date.getAs[String]("transaction_date")
    val transDate = LocalDate.parse(s"${transString.substring(0,4)}-${transString.substring(4,6)}-${transString.substring(6,8)}", formatter)
    val expireString = date.getAs[String]("membership_expire_date")
    val expireDate = LocalDate.parse(s"${expireString.substring(0,4)}-${expireString.substring(4,6)}-${expireString.substring(6,8)}", formatter)
    val isCancel = date.getAs[String]("is_cancel")

    if(isCancel == "1") {
      if(expireDate.isBefore(lastExpireDate)) {
        lastExpireDate = expireDate
      }
    } else {
      gap = ChronoUnit.DAYS.between(lastExpireDate, transDate).toInt
    }
  }
  gap
}

val data = spark
  .read
  .option("header", value = true)
  .csv("/mnt/kkbox/transactions/")

val historyCutoff = "20170131"

val historyData = data.filter(col("transaction_date")>="20170101" and col("transaction_date")<=lit(historyCutoff))
val futureData = data.filter(col("transaction_date") > lit(historyCutoff))

val calculateLastdayUDF = udf(calculateLastday _)
val userExpire = historyData
  .groupBy("msno")
  .agg(
    calculateLastdayUDF(
      collect_list(
        struct(
          col("payment_method_id"),
          col("payment_plan_days"),
          col("plan_list_price"),
          col("transaction_date"),
          col("membership_expire_date"),
          col("is_cancel")
        )
      )
    ).alias("last_expire")
  )

val predictionCandidates = userExpire
  .filter(
    col("last_expire") >= "20170201" and col("last_expire") <= "20170228"
  )
  .select("msno", "last_expire")


val joinedData = predictionCandidates
  .join(futureData,Seq("msno"), "left_outer")

val noActivity = joinedData
  .filter(col("payment_method_id").isNull)
  .withColumn("is_churn", lit(1))


val calculateRenewalGapUDF = udf(calculateRenewalGap _)
val renewals = joinedData
  .filter(col("payment_method_id").isNotNull)
  .groupBy("msno", "last_expire")
  .agg(
    calculateRenewalGapUDF(
      collect_list(
        struct(
          col("payment_method_id"),
          col("payment_plan_days"),
          col("plan_list_price"),
          col("transaction_date"),
          col("membership_expire_date"),
          col("is_cancel")
        )
      ),
      col("last_expire")
    ).alias("gap")
  )

val validRenewals = renewals.filter(col("gap") < 30)
  .withColumn("is_churn", lit(0))
val lateRenewals = renewals.filter(col("gap") >= 30)
  .withColumn("is_churn", lit(1))

val resultSet = validRenewals
  .select("msno","is_churn")
  .union(
    lateRenewals
      .select("msno","is_churn")
      .union(
        noActivity.select("msno","is_churn")
      )
  )

resultSet.write.format("delta").mode("overwrite").save("/mnt/kkbox/silver/train/")

UsageError: Cell magic `%%scala` not found.
